# 🧠 Clasificación de Frutas Frescas y Podridas con ResNet18


Este notebook implementa una solución de visión por computadora para la **clasificación de frutas frescas y podridas**, utilizando una arquitectura **ResNet18 preentrenada** sobre ImageNet. Se siguen los siguientes pasos:

1. **Importación de librerías** necesarias para procesamiento de datos, visualización, entrenamiento e inferencia.
2. **Descarga del dataset** directamente desde Kaggle utilizando `kagglehub`.
3. **Preprocesamiento de imágenes**, incluyendo redimensionamiento y normalización.
4. **Configuración del modelo**, congelando capas convolucionales y ajustando la capa final para clasificación específica.
5. **Entrenamiento y validación**, registrando pérdida y precisión por época.
6. **Visualización de métricas** para evaluar el rendimiento.
7. **Guardado del modelo** entrenado en disco.
8. **Fase de inferencia** sobre una imagen del conjunto de validación.


In [ ]:

# 📦 Importación de librerías necesarias
import kagglehub
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
from google.colab import drive
from PIL import Image  # Para manipular imágenes

print("✅ Librerías importadas correctamente")

try:
    # 📥 Descarga del dataset desde KaggleHub
    path = kagglehub.dataset_download("sriramr/fruits-fresh-and-rotten-for-classification")
    print("✅ Dataset descargado correctamente")
    print("Path to dataset files:", path)

    # ⚙️ Parámetros de configuración
    batch_size = 128  # Tamaño de batch para entrenamiento
    img_size = 224    # Tamaño de entrada esperado por ResNet18
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"🚀 Dispositivo de ejecución: {device}")

    # 🔄 Transformaciones para preprocesamiento de imágenes
    transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])  # Valores estándar de ImageNet
    ])
    print("✅ Transformaciones definidas")

    # 📁 Definición de rutas del dataset
    train_data_dir = os.path.join(path, 'dataset', 'train')
    val_data_dir = os.path.join(path, 'dataset', 'test')

    # 🧾 Carga de datasets con transformaciones
    train_dataset = datasets.ImageFolder(root=train_data_dir, transform=transform)
    val_dataset = datasets.ImageFolder(root=val_data_dir, transform=transform)
    class_names = train_dataset.classes

    print(f"🔍 Clases detectadas: {class_names}")
    print(f"📊 Imágenes de entrenamiento: {len(train_dataset)}")
    print(f"📊 Imágenes de validación: {len(val_dataset)}")

    # 🔁 Creación de DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # 🧠 Carga del modelo preentrenado
    model = models.resnet18(pretrained=True)
    print("🎯 Modelo ResNet18 cargado (preentrenado en ImageNet)")

    # ❄️ Congelar parámetros para usar como extractor de características
    for param in model.parameters():
        param.requires_grad = False

    # 🔄 Reemplazo de la capa fully-connected para ajuste fino
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, len(class_names))  # Ajustamos la última capa al número de clases
    model = model.to(device)
    print("✅ Modelo ajustado y enviado al dispositivo")

    # 🔧 Configuración del entrenamiento
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
    epochs = 10

    train_losses, val_losses, val_accuracies = [], [], []

    print("🚀 Comenzando entrenamiento...")

    # 🔁 Bucle de entrenamiento
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        train_losses.append(epoch_loss)

        # 🧪 Fase de validación
        model.eval()
        val_loss, correct, total = 0.0, 0, 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        accuracy = 100 * correct / total
        val_accuracies.append(accuracy)

        print(f"📈 Epoch {epoch+1}/{epochs} | Train Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | Accuracy: {accuracy:.2f}%")

    print("✅ Entrenamiento completado")

    # 📊 Visualización de resultados
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Entrenamiento')
    plt.plot(val_losses, label='Validación')
    plt.title("Curva de Pérdida")
    plt.xlabel("Épocas")
    plt.ylabel("Pérdida")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(val_accuracies, label='Precisión Validación', color='green')
    plt.title("Precisión por Época")
    plt.xlabel("Épocas")
    plt.ylabel("Precisión (%)")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # 💾 Guardado del modelo
    model_path = "modelo.pth"
    torch.save(model.state_dict(), model_path)
    print(f"✅ Modelo guardado como {model_path}")

    # 🔍 Inferencia sobre imagen de validación
    print("--- Fase de Inferencia ---")
    sample_image_path, _ = val_dataset.samples[0]
    image = Image.open(sample_image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        outputs = model(image)
        _, predicted_index = torch.max(outputs, 1)
    predicted_class = class_names[predicted_index.item()]
    print(f"🔎 Resultado de inferencia: {predicted_class}")

except Exception as e:
    print(f"❌ Error: {e}")
